In [2]:
import pandas as pd

'''
tug_war_model: 79.8
submission_weighted_vote_45: 80.3
submission_weighted_vote: 79.9
ensemble_submission: 79.8
submission_majority_vote_V3: 79.5
submission_majority_vote_V2: 80.2
submission_majority_vote: 80.5
submission_2: 80
submission_1: 79.8
'''



# List your submission files and a short model name for each
files_and_names = {
    'tug_war_model.csv': 'tug_war_model',
    'submission_weighted_vote_45.csv': 'weighted_vote_45',
    'submission_weighted_vote.csv': 'weighted_vote',
    'ensemble_submission.csv': 'ensemble',
    'submission_majority_vote_V3.csv': 'majority_vote_V3',
    'submission_majority_vote_V2.csv': 'majority_vote_V2',
    'submission_majority_vote.csv': 'majority_vote',
    'submission2.csv': 'submission2',
    'submission1.csv': 'submission1',
}

# Initialize empty DataFrame for merged data
merged_df = pd.DataFrame()

for filename, model_name in files_and_names.items():
    df = pd.read_csv(filename)
    
    # Normalize 'Transported' column to boolean if not already
    # If already bool or 0/1, this will work fine:
    df['Transported'] = df['Transported'].astype(bool)
    
    # Rename the Transported column to the model name
    df = df.rename(columns={'Transported': model_name})
    
    if merged_df.empty:
        merged_df = df.set_index('PassengerId')
    else:
        merged_df = merged_df.join(df.set_index('PassengerId'), how='outer')

# merged_df now has one column per model with boolean predictions indexed by PassengerId
print(merged_df.head())




             tug_war_model  weighted_vote_45  weighted_vote  ensemble  \
PassengerId                                                             
0013_01               True              True           True      True   
0018_01              False             False          False     False   
0019_01               True              True           True      True   
0021_01               True              True           True      True   
0023_01               True              True          False      True   

             majority_vote_V3  majority_vote_V2  majority_vote  submission2  \
PassengerId                                                                   
0013_01                  True              True           True         True   
0018_01                 False             False          False        False   
0019_01                  True              True           True         True   
0021_01                  True              True           True         True   
0023_01       

In [4]:
# Check if all predictions in each row are equal
all_agree = merged_df.nunique(axis=1) == 1  # True if only 1 unique value in the row

# Count how many rows have complete agreement
num_all_agree = all_agree.sum()
avg = num_all_agree/len(merged_df)

print(f"Number of passengers with full agreement across all models: {num_all_agree}")
print(f"Number of passengers with full agreement percentage: {avg:.3f}")


Number of passengers with full agreement across all models: 3882
Number of passengers with full agreement percentage: 0.908


In [16]:
import numpy as np

def dawid_skene(df, max_iter=100, tol=1e-5):
    """
    Dawid-Skene for binary labels.

    df: pd.DataFrame with shape (n_items, n_annotators), values 0/1.
    Returns: estimated true labels probabilities, annotator accuracies.
    """

    data = df.values
    n_items, n_annotators = data.shape

    # Initialize true label probabilities using majority vote
    p_true = data.mean(axis=1)

    # Initialize annotator accuracies to 0.8
    pi = np.full(n_annotators, 0.8)

    for iteration in range(max_iter):
        p_true_prev = p_true.copy()

        # E-step: estimate posterior probabilities of true label = 1
        # For each item, compute P(z_i=1 | labels, pi) proportional to:
        #   P(labels | z_i=1) * P(z_i=1)
        # Assuming P(z_i=1) ~ p_true[i] and independent annotators:

        likelihood_true = np.ones(n_items)
        likelihood_false = np.ones(n_items)

        for j in range(n_annotators):
            # P(label_j | z=1) = pi_j if label_j=1 else 1 - pi_j
            likelihood_true *= np.where(data[:, j] == 1, pi[j], 1 - pi[j])

            # P(label_j | z=0) = 1 - pi_j if label_j=1 else pi_j
            likelihood_false *= np.where(data[:, j] == 1, 1 - pi[j], pi[j])

        # Posterior probability of true label = 1 for each item:
        numerator = likelihood_true * p_true
        denominator = numerator + likelihood_false * (1 - p_true)
        p_true = numerator / denominator

        # M-step: update annotator accuracies
        for j in range(n_annotators):
            # Expected accuracy:
            # Sum over items: P(z=label_j) / total items
            correct_probs = p_true * (data[:, j] == 1) + (1 - p_true) * (data[:, j] == 0)
            pi[j] = correct_probs.sum() / n_items

        # Check convergence
        diff = np.abs(p_true - p_true_prev).max()
        if diff < tol:
            break

    # Final binary labels by threshold 0.5
    labels = p_true >= 0.5

    return labels, p_true, pi

# Example usage:
# Convert True/False to 1/0:
binary_df = merged_df.astype(int)

estimated_labels, prob_true, annotator_accs = dawid_skene(binary_df)

# Add results back to dataframe if you want:
df['estimated_label'] = estimated_labels
df['prob_true'] = prob_true

print("Annotator accuracies:", annotator_accs)
print("Annotator probs:", (df['prob_true'].sum())/ len(df))

Annotator accuracies: [0.97474865 0.97778817 0.97545008 0.98784195 0.97545008 0.97638532
 0.97708674 0.97919102 0.95955109]
Annotator probs: 0.534720598614876


In [18]:
submission = pd.DataFrame({
    'PassengerId': merged_df.index,               # or use df['id'] if it's a column
    'Transported': estimated_labels.astype(bool)  # convert 0/1 to True/False
})
submission.to_csv('dawid_skene_regression.csv', index=False)